In [1]:
import pandas as pd

In [37]:
df = pd.read_csv('Colleen_and_Alex_etl_status_copy_2.csv')

In [46]:
df = pd.read_csv('Colleen_and_Alex_etl_status.csv')

In [3]:
df.describe()

,Unnamed: 0,num_labels
count,21.000000,21.000000
mean,10.000000,1.285714
std,6.204837,0.956183
min,0.000000,0.000000
25%,5.000000,1.000000
50%,10.000000,1.000000
75%,15.000000,2.000000
max,20.000000,3.000000


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             1371 non-null   int64 
 1   url                    1371 non-null   object
 2   journal                1371 non-null   object
 3   get_paper_info_result  1371 non-null   object
 4   num_labels             1371 non-null   int64 
 5   error_traceback        222 non-null    object
dtypes: int64(2), object(4)
memory usage: 64.4+ KB


In [5]:
df['journal'].unique()

array(['royalsocietypublishing', nan, 'jeb', 'plos', 'pubmed'],
      dtype=object)

In [6]:
df.groupby(['journal']).size()

journal
jeb                       2
plos                      2
pubmed                    2
royalsocietypublishing    3
dtype: int64

In [7]:
df.groupby(['journal','url','get_paper_info_result'])['journal'].count()

journal                 url                                                                             get_paper_info_result
jeb                     https://jeb.biologists.org/content/213/2/288                                    success                  1
                        https://jeb.biologists.org/content/213/22/3920                                  success                  1
plos                    https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1002792  success                  2
pubmed                  https://pubmed.ncbi.nlm.nih.gov/14716016/                                       success                  1
                        https://pubmed.ncbi.nlm.nih.gov/20298484/                                       success                  1
royalsocietypublishing  https://royalsocietypublishing.org/doi/10.1098/rsbl.2004.0269                   success                  1
                        https://royalsocietypublishing.org/doi/10.1098/rsos.140322      

In [8]:
df['journal']

0     royalsocietypublishing
1     royalsocietypublishing
2                        NaN
3                        NaN
4                        NaN
5                        NaN
6                        NaN
7                        jeb
8                        NaN
9                        NaN
10                      plos
11                       NaN
12                    pubmed
13                       NaN
14                       NaN
15                    pubmed
16                      plos
17                       jeb
18    royalsocietypublishing
19                       NaN
20                       NaN
Name: journal, dtype: object

In [9]:
pd.__version__

'1.2.1'

In [14]:
df.value_counts('get_paper_info_result')

get_paper_info_result
no code to handle    10
success               9
error                 2
dtype: int64

In [13]:
type(df.value_counts('get_paper_info_result'))

pandas.core.series.Series

In [15]:
df.groupby('journal').value_counts('get_paper_info_result')

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_counts'

In [16]:
df.groupby('journal').count()

,Unnamed: 0,url,get_paper_info_result,num_labels,error_traceback
journal,,,,,
jeb,2,2,2,2,0
plos,2,2,2,2,0
pubmed,2,2,2,2,0
royalsocietypublishing,3,3,3,3,0


In [17]:
type(df.groupby('journal').count())

pandas.core.frame.DataFrame

In [ ]:
df.groupby('journal').value_counts('get_paper_info_result')

In [19]:
df.groupby('journal').get_paper_info_result.nunique()

journal
jeb                       1
plos                      1
pubmed                    1
royalsocietypublishing    1
Name: get_paper_info_result, dtype: int64

In [24]:
df.groupby('journal')['journal'].value_counts('num_labels')

journal                 journal               
jeb                     jeb                       1.0
plos                    plos                      1.0
pubmed                  pubmed                    1.0
royalsocietypublishing  royalsocietypublishing    1.0
Name: journal, dtype: float64

In [48]:
journal_groups = df.groupby('journal')

In [51]:
journal_groups.groups

<function PrettyDict.keys>

In [97]:
status_df = pd.DataFrame(columns=['journal','num_papers', 'success','error','no_code','no_labels'])

In [62]:
counts = journal_groups.get_group('nature')['get_paper_info_result'].value_counts()

In [66]:
'q' in counts

False

In [33]:
print(journal_groups)

In [98]:
for journal, group in journal_groups:
    #print(journal)
    result_values = group['get_paper_info_result'].value_counts()
    success = result_values.get("success", 0)
    error = result_values.get("error", 0)
    no_code = result_values.get("no_code", 0)
    no_labels = group.query('num_labels == 0').num_labels.count()
    num_papers = group.shape[0]
    #print(type(group))
    #print(result_values)
    status_df = status_df.append({'journal': journal, 'num_papers': num_papers, 'success': success, 'error': error, 'no_code': no_code, 'no_labels': no_labels}, ignore_index=True)

In [95]:
counts = journal_groups.get_group('nature').shape[0]
counts

94

In [80]:
journal_groups.get_group('nature')['num_labels']

AttributeError: 'Series' object has no attribute 'query'

In [100]:
status_df.sort_values(['num_papers'], ascending=[False])

,journal,num_papers,success,error,no_code,no_labels
109,sciencedirect,133,0,0,0,18
117,springer,127,0,127,0,20
61,jeb,127,93,34,0,19
76,nature,94,52,42,0,30
103,royalsocietypublishing,80,78,2,0,10
...,...,...,...,...,...,...
56,jb,1,0,0,0,0
55,jamanetwork,1,0,0,0,1
50,infona,1,0,0,0,1
48,geoscienceworld,1,0,0,0,1
